# <center>Procesado de Señales Fisiológicas</center>
## <center>Lab 2 Parte 2: Electroencefalograma (EEG)</center>
### <center>Rebeca Goya Esteban, Óscar Barquero Pérez </center>

Actualizado: 5 de marzo de 2025.

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Licencia de Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />Este obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">licencia de Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International</a>.  

# Parte 2: Señales reales

Esta segunda parte de la práctica tiene por objetivo el procesamiento y análisis de señales electroencefalográficas reales. 

Las señales con las que va a trabajar se registraron con el sistema:

- actiCAP Express: Sistema de Electrodos Activos Secos, se trata de un sistema de 32 canales.
- actiCHamp: amplificador, bateria y licencias. 
- BrainVision Recorder: software de adquisición de señales.
- BrainVision Analyzer: software que nos permitirá hacer una visualización y procesado previo, además de permitir exportar las señales a formato texto.

Para poder trabajar con las señales de EEG es necesario extraerlas de una serie de archivos binarios específicos del fabricante del equipo de adquisición (BrainVision). Cada registro dispone de 3 archivos diferentes:
* **Archivo .vhdr:** contiene las diferentes cabeceras necesarias para cargar las señales.
* **Archivo .vmrk:** contiene las marcas que se hayan podido introducir en el registro durante la adquisición.
* **Archivo .eeg:** contiene el registro completo.

## Configuración previa

Para facilitar el desarrollo de la práctica vamos a utilizar una toolbox de Python denominada [MNE-Python](https://mne.tools/stable/index.html), que permite realizar preprocesado y análisis de diferentes tipos de señales de origen cerebral, como puede ser el propio EEG, y otras como señales de MEG (magnetoencefalografía), sEEG (estereoencefalografía), o ECoG (electrocorticografía).

Para instalar este toolbox: `pip install mne`

## Cuestiones iniciales
Antes de cargar las señales, **abra con un editor de texto el archivo de cabeceras**, y describa algunos de los **parámetros** que nos van a ser de interés en la práctica, como pueden ser:
1. Número de canales.
2. Frecuencia de muestreo.
3. Resolución.

Piense en las siguientes cuestiones:
1. ¿Qué tipos de ruidos encontramos en las señales de EEG?
2. ¿Qué componentes espectrales están presentes en la actividad cerebral?
3. Respecto a los canales registrados, ¿se trata de señales monopolares o bipolares? ¿Por qué?

# Ejercicio 1.  Lectura y representación

En esta práctica **vamos a utilizar el MNE para realizar la carga directa de los registros** realizados con el software de BrainVision, ya que permite hacerlo de forma directa sin exportarlo previamente a texto. El código para realizar la carga de los registros es el siguiente:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import mne.io as io_eeg

#we are going to read an example eeg which is in the folder EEG_example. The method returns a RAW men object
filename = 'XXXX'
eeg_object = io_eeg.read_raw_brainvision(filename,preload = True) #the remaining parameters are left as default

La variable **_eeg_object_** es un objeto que dispone de una serie de métodos y atributos. En el caso de los atributos, los más importantes son:
* **ch_names:** Nombres de los canales.
* **n_times:** Número de instantes temporales.
* **times:** Vector de tiempo.
* **info: (dict)** [Información de medidas](https://mne.tools/stable/generated/mne.Info.html#mne.Info). Se trata de una estructura de datos que contiene los diferentes metadatos disponibles en el registro.

Respecto a los métodos, el más importante es **get_data()**, que nos permitirá acceder a las señales.

Una vez llegados a este punto, realice las siguientes tareas:
1. Imprima por pantalla los **nombres de los electrodos**.
2. Guarde en una variable (eeg) el **registro completo**. Guarde también el **vector de tiempo (t)** y calcule la **frecuencia de muestreo (fs) a partir del vector _t_**. Analice el tamaño de las matrices generadas.
3. **Represente 5 segundos del primer canal**, con el eje X en segundos. Pruebe a visualizar diferentes tramos de 10 segundos de la señal.
4. Trate de visualizar **intervalos más anchos y más estrechos de señal**, y **haga zoom sobre el segmento visualizado**.
5. **Visualice otro canal** de su interés.

In [ ]:
## 1. Print the electrodes' names.
print(XXXX)

## 2. Store both EEG signals and time vector on different variables, and compute the sampling frequency (fs).
eeg = XXXX
t = XXXX
fs = XXXX
print('EEG matrix shape: ', eeg.shape)
print('Times vector shape: ', t.shape)
print('Sampling rate (fs): %d Hz' %fs)

In [ ]:
## 3, 4 and 5. Different plots.
plt.figure()
ch_0 = XXXX
plt.plot(t,ch_0)
plt.xlim((40,43))
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (muV)')
plt.title('EEG channel')

Como puede comprobar, las señales son bastante ruidosas. Sin embargo, antes de filtrarlas, gastaremos algo de tiempo revisando los diferentes métodos de estimación espectral.

# Ejercicio 2.  Interpretación de la DEP de una señal de EEG

**Calcule y represente** la estimación de la DEP de un canal de EEG utilizando el método de Welch. Después, trate de interpretar dicha estimación, y justifique si la señal de EEG necesita ser filtrada o no (de ser así, ¿qué filtrado aplicaría?).

In [ ]:
from scipy import signal

ch_0 = XXXX  # Let's choose a single EEG channel (the first one, for example).
f_welch_eeg, Px_welch_eeg = signal.welch(XXXX, fs=fs, window='hamming', nperseg=512, nfft=1024)

plt.figure()
plt.plot(f_welch_eeg,10*np.log10(Px_welch_eeg))
plt.xlabel('Frequency (Hz)')
plt.xlim(0,160)
plt.ylabel('Magnitude (dB)')
plt.title('Welch\'s method (Original EEG)')

# Ejercicio 3.  Filtrado

Las señales que ha visualizado en el ejercicio anterior son **señales "crudas (raw)"**, por lo que es necesario realizar algún tipo de pre-procesado previo al análisis, con el objetivo de eliminar ruido. Para ello, además de **numpy**, se va a utilizar el paquete _signal_ de **_scipy_**, ya que contiene un amplio abanico de funciones para procesado de señal. Puede consultar la documentación en la siguiente URL:

https://docs.scipy.org/doc/scipy/reference/signal.html

A continuación, realice los siguientes pasos de pre-procesado:

1. **Aplique un filtro adaptado "notch" sobre la señal de EEG original para cancelar el ruido de 50 Hz** (puede usar $\mathcal{signal.iirnotch}$ y $\mathcal{signal.filtfilt}$), vuelva a representar la señal y compruebe en qué medida se ha eliminado este tipo de ruido. Puede probar en diferentes canales. **¿Por qué aplicamos un filtro a esta frecuencia? ¿Existen casos en los que no se deba filtrar a esa frecuencia?**
2. **Aplique un filtro FIR pasobanda entre 0.5 y 40 Hz sobre la señal de EEG original** (puede usar $\mathcal{signal.firwin}$ y $\mathcal{signal.filtfilt}$). Aplique este mismo filtrado **sobre la señal filtrada con el notch**, y compare los resultados. **¿Existen diferencias entre las señales? ¿Por qué se escogen estas frecuencias de corte?**
3. **Aplique un filtro IIR pasobanda entre 0.5 y 40 Hz sobre la señal de EEG original** (puede usar $\mathcal{signal.butter}$ y $\mathcal{signal.filtfilt}$). Aplique este mismo filtrado **sobre la señal filtrada con el notch**, y compare los resultados. **¿Cuáles son las principales diferencias entre las señales filtradas con el filtro IIR y con el filtro FIR?**
4. **Represente las respuestas en frecuencia de los filtros diseñados** utilizando $\mathcal{signal.freqz}$.
5. **Calcule y represente** la DEP de un canal de la señal de EEG filtrada (filtro FIR pasobanda). **¿Considera que el ruido se ha eliminado corretamente?**

In [ ]:
## 1. Design and apply a notch filter.
f0 = XXXX     # Frequency to be removed from the signal (Hz)
Q = 30.0    # Quality factor

b_notch, a_notch = signal.iirnotch(XXXX, XXXX, XXXX)   # Create notch filter
eeg_notch = signal.filtfilt(XXXX, XXXX, XXXX)   # Apply filter

plt.figure()
plt.plot(t,eeg[0,:])        # Plot first channel (original)
plt.plot(t,eeg_notch[0,:])        # Plot first channel (notch-filtered)
plt.xlim((50,55))
plt.xlabel('Time [s]')
plt.ylabel('Amplitude [uV]')
plt.legend(('Original EEG','Notch-filtered EEG'))
plt.title('Comparison between original EEG and Notch-filtered EEG')

## 2. Design and apply a FIR bandpass filter.
f_low = XXXX        # Low cutoff frequency
f_high = XXXX        # High cutoff frequency
numtaps = 64      # Filter length

b_bp_FIR=signal.firwin(numtaps, [f_low, f_high], pass_zero=False,fs=fs)     # Create FIR bandpass filter

eeg_nt_bandpass_FIR = signal.filtfilt(XXXX, 1, XXXX)        # Apply filter (notch-filtered signals)
eeg_orig_bandpass_FIR = signal.filtfilt(XXXX, 1, XXXX)            # Apply filter (original signals)

## Plot comparison between signals (FIR-filtered)
plt.figure()
plt.plot(t,eeg_nt_bandpass_FIR[0,:])     #Plot first channel (after band-pass filtering)
plt.plot(t,eeg_orig_bandpass_FIR[0,:])     #Plot first channel (after notch + band-pass filtering)
plt.xlim((60,65)),
plt.ylim((0.04,0.08)),
plt.legend(('BP-filtered EEG','Notch+BP filtered EEG'))
plt.title('Comparison between BP-filtered EEG and Notch+BP filtered EEG (FIR)')

## 3. Design and apply an IIR bandpass filter.
f_low = XXXX      # Low cutoff frequency
f_high = XXXX        # High cutoff frequency
filter_order = 5       # Filter order

b_bp_IIR, a_bp_IIR=signal.butter(filter_order, [f_low, f_high], btype='bandpass',fs=fs)     # Create IIR bandpass filter
eeg_nt_bandpass_IIR = signal.filtfilt(XXXX, XXXX, XXXX)        # Apply filter (notch-filtered signals)
eeg_orig_bandpass_IIR = signal.filtfilt(XXXX, XXXX, XXXX)            # Apply filter (original signals)

## Plot comparison between signals (IIR-filtered)
plt.figure()
plt.plot(t,eeg_orig_bandpass_IIR[0,:])     #Plot first channel (after band-pass filtering)
plt.plot(t,eeg_nt_bandpass_IIR[0,:])     #Plot first channel (after notch + band-pass filtering)
plt.xlim((50,55)),

plt.legend(('BP-filtered EEG','Notch+BP filtered EEG'))
plt.title('Comparison between BP-filtered EEG and Notch+BP filtered EEG (IIR)')

In [ ]:
## 4. Plot frequency response of the filters
w_notch, h_notch = signal.freqz(XXXX, XXXX, fs=fs)     # Frequency response of the notch filter
w_bp_FIR, h_bp_FIR = signal.freqz(XXXX, fs=fs)                       # Frequency response of the bandpass filter (FIR)
w_bp_IIR, h_bp_IIR = signal.freqz(XXXX, XXXX, fs=fs)     # Frequency response of the notch filter

plt.figure(figsize=[15,5])
ax1 = plt.subplot(131)
ax1.set_title('Notch filter frequency response')
ax1.plot(w_notch, 20 * np.log10(abs(h_notch)), 'b')
ax1.set_xlabel('Frequency [Hz]')
ax1.set_ylabel('Amplitude [dB]', color='b')
ax2 = ax1.twinx()
angles = np.unwrap(np.angle(h_notch))
ax2.plot(w_notch, angles, 'g')
ax2.set_ylabel('Angle (radians)', color='g')
ax2.grid()
ax2.axis('tight')
plt.xlim((0,100))

ax3 = plt.subplot(132)
plt.ylim((-150,10))
ax3.set_title('Bandpass filter frequency response (FIR)')
ax3.plot(w_bp_FIR, 20 * np.log10(abs(h_bp_FIR)), 'b')
ax3.set_xlabel('Frequency [Hz]')
ax3.set_ylabel('Amplitude [dB]', color='b')
ax4 = ax3.twinx()
angles = np.unwrap(np.angle(h_bp_FIR))
ax4.plot(w_bp_FIR, angles, 'g')
ax4.set_ylabel('Angle (radians)', color='g')
ax4.grid()
ax4.axis('tight')
plt.xlim((0,100))

ax5 = plt.subplot(133)
plt.ylim((-150,10))
ax5.set_title('Bandpass filter frequency response (IIR)')
ax5.plot(w_bp_IIR, 20 * np.log10(abs(h_bp_IIR)), 'b')
ax5.set_xlabel('Frequency [Hz]')
ax5.set_ylabel('Amplitude [dB]', color='b')
ax6 = ax5.twinx()
angles = np.unwrap(np.angle(h_bp_IIR))
ax6.plot(w_bp_IIR, angles, 'g')
ax6.set_ylabel('Angle (radians)', color='g')
ax6.grid()
ax6.axis('tight')
plt.xlim((0,100))

plt.tight_layout()

In [ ]:
## 5. Plot the PSD of one filtered signal (FIR filtered)
f_welch_eeg_filtered, Px_welch_eeg_filtered = signal.welch(XXXX, fs=fs, window='hamming', nperseg=512, nfft=1024)

plt.figure()
plt.plot(f_welch_eeg_filtered,10*np.log10(Px_welch_eeg_filtered))
#plt.plot(f_welch_eeg_filtered,Px_welch_eeg_filtered)
plt.xlim(0,160)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
plt.title('Welch\'s method (Original EEG)')

# Ejercicio 4. Cálculo de potencia media utilizando la DEP

Una forma de evaluar si, en la realización de un experimento de EEG, se produce algún tipo de cambio en la señal resultante, es mediante el cálculo de **parámetros espectrales**.

En los registros de EEG realizados en la práctica, se ha realizado el siguiente protocolo:
1. Registro de 30 segundos con los ojos abiertos.
2. Registro de 30 segundos con los ojos cerrados.

Un tipo de ritmo típico que puede aparecer en la señal de EEG con ojos cerrados son las denominadas **ondas alfa**, que abarcan un rango espectral desde 8 hasta 13Hz. Para esta parte de la práctica se pide lo siguiente **(utilizando el registro hecho y filtrado)**:
1. **Calcule el Periodograma de Welch de cada segmento** utilizando un canal occipital (señales filtradas con el filtro FIR) y los mismos parámetros que en el Ejercicio 2.
2. Utilizando la estimación de la DEP anterior, **calcule la potencia media en cada uno de los segmentos en la banda alfa y el ratio respecto a la potencia media total**, y reflexione sobre los resultados. Para calcular la potencia media, puede utilizar un método de integración numérica, como *scipy.integrate.simpson*.

In [ ]:
from scipy.integrate import simpson

## Select one occipital channel
oc_eeg = XXXX  

## Compute Welch's Periodogram for each segment 
##0-30 sg

f_welch_eeg, Px_welch_eeg_oc1 =signal.welch(oc_eeg[0:int(30*fs)], fs=fs, window='hamming', nperseg=512, nfft=1024)
##30-60 sg
_, Px_welch_eeg_oc2 =signal.welch(XXXX, fs=fs, window='hamming', nperseg=512, nfft=1024)

## Compute the total power for each segment.
idx_delta_total = np.logical_and(f_welch_eeg >= 0.5, f_welch_eeg <= 50)
avgp_total_1 = simpson(Px_welch_eeg_oc1[idx_delta_total],f_welch_eeg[idx_delta_total])    # 1st segment
avgp_total_2 = simpson(Px_welch_eeg_oc2[idx_delta_total],f_welch_eeg[idx_delta_total])    # 2nd segment
## Compute the  power on the alpha band for each segment.
f_low_index = (np.abs(f_welch_eeg-8)).argmin()    # Compute the index that corresponds (or is close) to f=8 Hz
f_high_index = (np.abs(f_welch_eeg-13)).argmin()  # Compute the index that corresponds (or is close) to f=13 Hz


# Average power on the alpha band for the 1st segment.
avgp_1 = simpson(Px_welch_eeg_oc1[f_low_index:f_high_index],f_welch_eeg[f_low_index:f_high_index])     
r_seg1 = avgp_1/avgp_total_1*100;   # Alpha-band power vs total average power ratio for the 1st segment.

avgp_2 = simpson(Px_welch_eeg_oc2[f_low_index:f_high_index],f_welch_eeg[f_low_index:f_high_index])     
r_seg2 = avgp_2/avgp_total_2*100;   # Alpha-band power vs total average power ratio for the 1st segment.
                                    
print('Alpha band power ratio (segment 1): ', r_seg1)
print('Alpha band power ratio (segment 2): ', r_seg2)